In [1]:
import pandas as pd
import random

In [2]:
results = pd.read_csv('results.csv')
results.loc[:, 'mask'] = results.loc[:, 'mask'].str.replace('X', '')
roster = pd.read_csv('roster.csv').set_index('idplayer')
tournaments = pd.read_csv('tournaments.csv').set_index('idtournament')
tournaments = tournaments.loc[~tournaments.questions_total.isnull(), :]
common_tours = list(set(tournaments.index).intersection(set(results.idtournament)))
results = results.loc[results.idtournament.isin(common_tours), :]
tournaments = tournaments.loc[common_tours, :]

q_total = pd.DataFrame(results.groupby('idtournament').first()\
                       .loc[:, 'mask'].str.len()).dropna().rename(columns={'mask': 'questions_total'})
tournaments = pd.merge(tournaments.drop(columns=['questions_total']), q_total, left_index=True, right_index=True)

In [3]:
tournaments.head(1)

,name,town,long_name,date_start,date_end,tour_count,tour_questions,tour_ques_per_tour,type_name,main_payment_value,discounted_payment_value,discounted_payment_reason,date_requests_allowed_to,comment,site_url,main_payment_currency,discounted_payment_currency,questions_total
idtournament,,,,,,,,,,,,,,,,,,
2,Летние зори,Саранск,NaN,2003-08-09 00:00:00,2003-08-09 00:00:00,3.0,15.0,NaN,Обычный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0


In [4]:
results.head(2)

,idteam,current_name,base_name,position,questions_total,mask,bonus_a,bonus_b,tech_rating,predicted_position,d_bonus_a,d_bonus_b,d_diff_bonus,included_in_rating,idtournament,diff_bonus
0,209,Команда Коваленко,Команда Коваленко,11.0,34,1111101000101111001100011000111101001111001110...,2230.0,-87.0,4509.0,7.0,2230,663,-87.0,1.0,3236,NaN
1,270,Синоп,Синоп,14.5,31,1110110000101100001100101101101101010011010000...,2189.0,119.0,2567.0,20.0,2189,618,119.0,1.0,3236,NaN


In [5]:
roster.head(1)

,is_captain,is_base,is_foreign,idteam,idtournament
idplayer,,,,,
19298,1,1,0,209,3236


In [6]:
def skill_init():
    return random.uniform(0, 10)
roster.loc[:, 'skill'] = None
roster.loc[:, 'skill'] = roster.loc[:, 'skill'].apply(lambda x: skill_init())
roster.head(1)
# player_skills_dict = {id: skill_init() for id in roster.idplayer}
# pd.DataFrame(player_skills_dict, index=[0]).T.head()

,is_captain,is_base,is_foreign,idteam,idtournament,skill
idplayer,,,,,,
19298,1,1,0,209,3236,0.981719


In [7]:
def difficulty_init():
    return random.uniform(0, 10)
question_difficulty = {f"{str(tourn)}_{str(ques)}": difficulty_init() for tourn in tournaments.index 
                       for ques in range(int(tournaments.loc[tourn, 'questions_total']))}
pd.DataFrame(question_difficulty, index=[0]).T.head()

,0
2_0,2.699001
2_1,1.127561
2_2,7.292661
2_3,5.025678
2_4,0.829616


In [8]:
def sigmoid(x):
    return 1 / (1 + pd.np.exp(-x))    

**Model**<br>
$p(y=1| S_{team}, \theta) = \sigma(S_{team} - \theta)$ <br>
$S_{team}$ - team skill<br>
$S_{team} = S_{player\_1} + S_{player\_2} + ... + S_{player\_N}$<br>
$\theta_j$ - difficulty of question j<br>
y - team answer, 1 for correct, 2 for incorrect<br>
$p(y_{hat}=y_{real}| S_{team}, \theta) = \sigma(S_{team} - \theta)^{y_{real}} * (1-\sigma(S_{team} - \theta))^{(1-y_{real})}$<br>
<br>
$LikelyHood = \frac{1}{N}\prod_{i=1}^{N} p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta)$<br>
$log(LikelyHood) = LL = \frac{1}{N}\sum_{i=1}^{N} log(p(y_{hat\_i}=y_{real\_i}| S_{team}, \theta))=
y_{real}*log(\sigma(S_{team} - \theta)) + (1-y_{real})* log(1-\sigma(S_{team} - \theta))$<br>
<br>
$-log(LikelyHood) \rightarrow min$ w.r.t. $S_{player\_i}$ and $\theta_j$

**Derivatives:**

$\sigma'(x) = \sigma(x)*(1-\sigma(x))$

$\frac{\partial LL}{\partial \theta} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) * (-1) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) * (-1) = 
-y_{real}*(1-\sigma(S_{team} - \theta)) + (1-y_{real})*\sigma(S_{team} - \theta)
$<br>

$\frac{\partial LL}{\partial S_i} = \frac{y_{real}}{\sigma(S_{team} - \theta)} * \sigma'(S_{team} - \theta) +
\frac{1-y_{real}}{1-\sigma(S_{team} - \theta)} * (-\sigma'(S_{team} - \theta)) = 
y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)$<br>


**Short derivatives:**

$\frac{\partial -LL}{\partial \theta} = +y_{real}*(1-\sigma(S_{team} - \theta)) - (1-y_{real})*\sigma(S_{team} - \theta)
$<br>

$\frac{\partial -LL}{\partial S_i} = -y_{real}*(1-\sigma(S_{team} - \theta)) + (1-y_{real})*\sigma(S_{team} - \theta)$<br>


**Gradient update**<br>
$S_{i\_new} = S_{i\_old} - learning\_rate * \frac{\partial LL}{\partial S_i}$<br>
$\theta_{new} = \theta_{old} - learning\_rate * \frac{\partial LL}{\partial \theta}$<br>

In [9]:
from tqdm import tqdm

In [14]:
def gd(lr=0.01, epochs=10, steps_per_tour = 10):
    fails=0
    fail_list = []
    for _ in tqdm(range(epochs)):
        # get random tournament, team and question
#         tour = random.sample(results.idtournament.unique().tolist(), 1)[0]
        tour = random.sample(tournaments.index.tolist(), 1)[0]
        t_subset = results.loc[results.idtournament == tour, ['idteam', 'mask']]
#         for _ in tqdm(range(steps_per_tour)):
        for _ in range(steps_per_tour):
            team = random.sample(t_subset.idteam.tolist(), 1)[0]
            players = roster.loc[(roster.idteam == team) & (roster.idtournament == tour), :].index.tolist()
            quest = random.randint(0, int(tournaments.loc[tour, 'questions_total'])-1)

            # get question difficulty
            quest_dif_key = f"{str(tour)}_{str(quest)}"
            try:
                difficulty = question_difficulty[quest_dif_key]
            except:
                fails+=1
                fail_list.append(quest_dif_key)
                continue

            # calculate team skill and sigmoid
#             skills = [player_skills_dict[player] for player in players]
            skills = roster.loc[(roster.index.isin(players)) &
                                (roster.idteam == team) &
                                (roster.idtournament == tour), 'skill'].tolist()
            team_skill = sum(skills)
            team_sigmoid = sigmoid(team_skill - difficulty)

            # get y (1 for correct answer, 0 for incorrect)
            y_true = int(t_subset.loc[t_subset.idteam == team, 'mask'].values[0][quest])

            # weights update
            question_difficulty[quest_dif_key] -= lr * (y_true * (1-team_sigmoid) - (1-y_true)*team_sigmoid)
            roster.loc[players, ]
#             for player in players:
#                 player_skills_dict[player] -= lr * (-y_true * (1-team_sigmoid) + (1-y_true)*team_sigmoid)
    print(f"Fails: {fails}")
    return fail_list

In [15]:
fl = gd(epochs=10000, steps_per_tour = 100)

  0%|          | 0/10000 [00:00<?, ?it/s]


NameError: name 'player_skills_dict' is not defined

In [ ]:
fl


In [ ]:
question_difficulty['450_13']

In [ ]:
results.loc[results.idtournament==450, 'mask'].str[0]